### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [223]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', 
                     delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], 
                     dtype={'movie_id': object}, engine='python', encoding='utf-8')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', 
                      delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], 
                      dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python', encoding='utf-8')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [224]:
from IPython.display import display

# Initial overview of the data
display(movies.head(5))
display(reviews.head(5))

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


,user_id,movie_id,rating,timestamp
0,1,0111161,10,1373234211
1,1,0117060,7,1373415231
2,1,0120755,6,1373424360
3,1,0317919,6,1373495763
4,1,0454876,10,1373621125


In [225]:
display(movies.describe())
display(reviews.describe())

,movie_id,movie,genre
count,32885,32885,32612
unique,32885,32834,1241
top,3813916,Home (2016),Drama
freq,1,4,3750


,rating
count,761230.000000
mean,7.298474
std,1.854598
min,0.000000
25%,6.000000
50%,7.000000
75%,9.000000
max,10.000000


In [226]:
movies.isna().sum()

movie_id      0
movie         0
genre       273
dtype: int64

In [227]:
reviews.isna().any()

user_id      False
movie_id     False
rating       False
timestamp    False
dtype: bool

#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [228]:
import re

# Create new date column
movies['date'] = movies.apply(lambda row: int(re.match('.*\((.*)\)', row.movie).groups()[0]), axis=1)

In [229]:
from functools import partial

# Create century columns
is_greater_than = lambda century, row: int(row.date >= century)

movies['2000s'] = movies.apply(partial(is_greater_than, 2000), axis=1)
movies['1900s'] = movies.apply(partial(is_greater_than, 1900), axis=1)
movies['1800s'] = movies.apply(partial(is_greater_than, 1800), axis=1)

assert movies[['1800s', '1900s', '2000s']].any(axis=1).all()
movies.head(5)

,movie_id,movie,genre,date,2000s,1900s,1800s
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,0,0,1
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,0,0,1
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,0,0,1
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,0,0,1
4,0000091,Le manoir du diable (1896),Short|Horror,1896,0,0,1


In [239]:
# Collect all genres
all_genres = []
[all_genres.extend(g) for g in 
     map(lambda x: x.split('|'), 
         filter(lambda x: type(x) != float, movies.genre))]
all_genres = set(all_genres)
print('Number of different genres: {}'.format(len(all_genres)))
all_genres

Number of different genres: 28


{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

In [231]:
# Add dummy columns for genres
for genre in all_genres:
    movies[genre.lower()] = movies.genre.apply(lambda x: 0 if type(x) is float else 
                             1 if genre in x.split('|') else 0)
    
movies.head(3)

,movie_id,movie,genre,date,2000s,1900s,1800s,game-show,comedy,action,...,short,talk-show,war,drama,western,sport,animation,musical,reality-tv,crime
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [234]:
import datetime

# Convert timestamp to date
change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [241]:
# Save both clean data frames
movies.to_csv('data/movies_clean.csv')
reviews.to_csv('data/reviews_clean.csv')